In [1]:
from gurobipy import *

In [2]:
x = [ [0,1], [1, 2], [3, 4]]
y = [ [1,1], [2,2], [4,4]]
gamma = 0.1

In [5]:
def optimize(x, y, gamma):
    
    m = Model()
    a = [ m.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY, name="a0"),
          m.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY, name="a1") ]
    beta = m.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY, name="beta")

    u = []
    N = len(x)
    for i in range(N):
        u.append(m.addVar(name=("u%d" % i)))

    v = []
    M = len(y)
    for i in range(M):
        v.append(m.addVar(name=("v%d" % i)))

    m.update()

    m.setObjective(a[0]*a[0] + a[1]*a[1] + gamma*(quicksum(u) + quicksum(v)))

    for i in range(N):
        xi = x[i]
        m.addConstr(a[0]*xi[0] + a[1]*xi[1] - beta >= 1 - u[i], name=("x%d" % i))

    for i in range(M):
        yi = y[i]
        m.addConstr(a[0]*yi[0] + a[1]*yi[1] - beta <= -(1 - v[i]), name=("y%d" % i))

    m.update()
    m.write('test.lp')
    m.optimize()

    results = {}
    if m.Status == GRB.OPTIMAL:
        results['a'] = [a[0].X, a[1].X]
        results['beta'] = beta.X

    return results

In [6]:
optimize(x,y,gamma)

Optimize a model with 6 rows, 9 columns and 23 nonzeros
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e-01, 1e-01]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 0 columns
Presolve time: 0.02s
Presolved: 5 rows, 9 columns, 24 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 2
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   7.00300000e+02 -5.22959078e-14  0.00e+00 1.79e-07  1.00e+06     0s
   1   7.00205406e+02  2.91923223e-01  0.00e+00 1.72e-10  1.02e+03     0s
   2   6.16099193e+02  2.94335784e-01  0.00e+00 1.73e-16  5.13e+01     0s
   3   1.29166828e+00  2.94773254

{'a': [-0.1499999981611477, 1.3162046728137371e-09],
 'beta': -0.32036333384010107}